In [33]:

# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
import scipy.stats as stats
from scipy.stats import chi2_contingency, ttest_ind

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")


# Explorando CSV para línea del tiempo de música rock

In [81]:
# cargamos el dataframe correspondiente 
df = pd.read_csv("archivos/UltimateClassicRock.csv")

display(df.head())

,Track,Artist,Album,Year,Duration,Time_Signature,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Popularity
0,Play A Simple Song,38 Special,38 Special,1977,3:13,4,0.521,0.367,0,-13.866,1,0.0278,0.6920,0.000003,0.1080,0.789,83.412,16
1,Four Wheels,38 Special,38 Special,1977,4:43,4,0.535,0.710,2,-12.287,1,0.0428,0.0100,0.023000,0.0495,0.445,160.361,10
2,Fly Away,38 Special,38 Special,1977,5:13,4,0.563,0.563,2,-10.781,1,0.0263,0.0357,0.001850,0.1400,0.564,106.739,13
3,Tell Everybody,38 Special,38 Special,1977,4:09,4,0.638,0.694,11,-10.206,0,0.0310,0.1610,0.000034,0.0908,0.936,124.962,10
4,Just Wanna Rock & Roll,38 Special,38 Special,1977,5:57,4,0.388,0.701,2,-9.984,1,0.0360,0.0130,0.042200,0.1150,0.769,126.769,11


In [87]:
df.columns

Index(['Track', 'Artist', 'Album', 'Year', 'Duration', 'Time_Signature',
       'Danceability', 'Energy', 'Key', 'Loudness', 'Mode', 'Speechiness',
       'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo',
       'Popularity'],
      dtype='object')

In [82]:
def exploracion(df):
    df_info = pd.DataFrame()
    df_info["% nulos"] = round(df.isna().sum()/df.shape[0]*100, 2).astype(str)+"%"
    df_info["% no_nulos"] = round(df.notna().sum()/df.shape[0]*100, 2).astype(str)+"%"
    df_info["tipo_dato"] = df.dtypes
    df_info["num_valores_unicos"] = df.nunique()
    print(f"""El DataFrame tiene {df.shape[0]} filas y {df.shape[1]} columnas.
Tiene {df.duplicated().sum()} datos duplicados, lo que supone un porcentaje de {round(df.duplicated().sum()/df.shape[0], 2)}% de los datos.
Hay {len(list(df_info[(df_info["% nulos"] != "0.0%")].index))} columnas con datos nulos, y son:
{list(df_info[(df_info["% nulos"] != "0.0%")].index)}
y sin nulos hay {len(list(df_info[(df_info["% nulos"] == "0.0%")].index))} columnas y son:
{list(df_info[(df_info["% nulos"] == "0.0%")].index)}
A continuación tienes un detalle sobre los datos nulos y los tipos y número de datos:""")
    display(df_info.head())
    print("Principales estadísticos de las columnas categóricas:")
    display(df.describe(include="O").T)
    print("Principales estadísticos de las columnas numéricas:")
    display(df.describe(exclude="O").T)
    return df_info

exploracion(df)

El DataFrame tiene 14418 filas y 18 columnas.
Tiene 0 datos duplicados, lo que supone un porcentaje de 0.0% de los datos.
Hay 0 columnas con datos nulos, y son:
[]
y sin nulos hay 18 columnas y son:
['Track', 'Artist', 'Album', 'Year', 'Duration', 'Time_Signature', 'Danceability', 'Energy', 'Key', 'Loudness', 'Mode', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo', 'Popularity']
A continuación tienes un detalle sobre los datos nulos y los tipos y número de datos:


,% nulos,% no_nulos,tipo_dato,num_valores_unicos
Track,0.0%,100.0%,object,13328
Artist,0.0%,100.0%,object,94
Album,0.0%,100.0%,object,1247
Year,0.0%,100.0%,int64,63
Duration,0.0%,100.0%,object,911


Principales estadísticos de las columnas categóricas:


,count,unique,top,freq
Track,14418,13328,Start Me Up - Live,8
Artist,14418,94,Bob Dylan,454
Album,14418,1247,Fleetwood Mac,35
Duration,14418,911,4:13,102


Principales estadísticos de las columnas numéricas:


,count,mean,std,min,25%,50%,75%,max
Year,14418.0,1987.634693,15.318819,1962.0,1975.000000,1983.000000,1999.000000,2024.000
Time_Signature,14418.0,3.917811,0.356628,0.0,4.000000,4.000000,4.000000,5.000
Danceability,14418.0,0.503063,0.142619,0.0,0.405000,0.509000,0.603000,0.987
Energy,14418.0,0.656563,0.229607,0.0,0.493000,0.690500,0.854000,0.998
Key,14418.0,5.166597,3.503423,0.0,2.000000,5.000000,9.000000,11.000
Loudness,14418.0,-9.438675,4.179623,-60.0,-11.914250,-8.810500,-6.341500,-0.203
Mode,14418.0,0.730129,0.443908,0.0,0.000000,1.000000,1.000000,1.000
Speechiness,14418.0,0.051354,0.046291,0.0,0.031800,0.039000,0.054300,0.952
Acousticness,14418.0,0.226924,0.268857,0.0,0.013100,0.104000,0.370000,0.995
Instrumentalness,14418.0,0.089682,0.215783,0.0,0.000013,0.000737,0.028375,0.992


,% nulos,% no_nulos,tipo_dato,num_valores_unicos
Track,0.0%,100.0%,object,13328
Artist,0.0%,100.0%,object,94
Album,0.0%,100.0%,object,1247
Year,0.0%,100.0%,int64,63
Duration,0.0%,100.0%,object,911
Time_Signature,0.0%,100.0%,int64,5
Danceability,0.0%,100.0%,float64,788
Energy,0.0%,100.0%,float64,1075
Key,0.0%,100.0%,int64,12
Loudness,0.0%,100.0%,float64,8993


## Valores unicos de las variables categoricas ✨

In [83]:
# creamos una lista con los nombres de las columnas categoricas 
columnas = df.select_dtypes(include='object').columns.tolist()
print(columnas)
# empezamos a iterar por cada una de las columnas para sacar sus valores únicos y sus frecuencias
for columna in columnas:
    print(f" \n----------- ESTAMOS ANALIZANDO LA COLUMNA: '{columna.upper()}' -----------\n")
    print(f"Sus valores únicos son: {df[columna].unique()}\n")
    print(f"Las frecuencias de los valores únicos de las categorías son: {df[columna].value_counts()} ")

['Track', 'Artist', 'Album', 'Duration']
 
----------- ESTAMOS ANALIZANDO LA COLUMNA: 'TRACK' -----------

Sus valores únicos son: ['Play A Simple Song' 'Four Wheels' 'Fly Away' ... 'Have A Little Mercy'
 "Flyin' High" 'Heartache In Blue']

Las frecuencias de los valores únicos de las categorías son: Track
Start Me Up - Live         8
Tumbling Dice - Live       8
Honky Tonk Women - Live    8
Tonight                    7
Without You                7
                          ..
Four Little Diamonds       1
Rock 'N' Roll Is King      1
Without Someone            1
Sorrow About to Fall       1
Heartache In Blue          1
Name: count, Length: 13328, dtype: int64 
 
----------- ESTAMOS ANALIZANDO LA COLUMNA: 'ARTIST' -----------

Sus valores únicos son: ['38 Special' 'ABBA' 'Aerosmith' 'Air Suppy' 'Alice Cooper'
 'Allman Brothers' 'America' 'Bachman-Turner Overdrive' 'Bad Company'
 'Bee Gees' 'Billy Joel' 'Black Sabbath' 'Blondie' 'Blue Oyster Cult'
 'Bob Dylan' 'Bob Seger' 'Bon Jovi' 'Bos

## DUPLICADOS

In [84]:
get_duplicate_rows(df)

,Track,Artist,Album,Year,Duration,Time_Signature,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Popularity


In [43]:
# index=False es útil si solo quieres los datos y no necesitas la columna de índice.
#df.to_csv("archivos/informacion_artista_MOD.csv", index=False)

### Explorando otro archivo de música electrónica para ver de cuántos años hay datos.

In [44]:
# cargamos el dataframe correspondiente 
df2 = pd.read_csv("archivos/Electro_music_on_Spotify.csv")

display(df.head())

,artist,title,label,country,format,release_date,genre,styles,have,want,num_ratings,average_rating,lowest_price,median_price,highest_price
0,Subterfuge,The Foundation Series Volume One,Visillusion,US,Vinyl,1997,Electronic,"House,Techno,Electro",93,423,31,3.81,$2.00,$39.02,$86.96
1,Titiyo,My Body Says Yes,Arista,UK,Vinyl,1991-04-01,Electronic,House,136,30,11,4.36,$0.43,$1.88,$5.43
2,Mariah Carey,Joy To The World,Columbia,US,Vinyl,1994-11,Electronic,"House,Garage House,Holiday",75,106,5,4.4,$1.99,$16.29,$33.71
3,Rhythmstate,Everybody,Nitebeat,US,Vinyl,1997,Electronic,"House,Breakbeat",22,57,6,4,$2.00,$7.00,$25.00
4,Exposé,"Stop, Listen, Look & Think",Arista,US,Vinyl,1990,Electronic,House,115,19,12,3.83,$0.79,$1.50,$4.34


In [ ]:
df2.columns

In [46]:
exploracion(df2)

El DataFrame tiene 57461 filas y 50 columnas.
Tiene 0 datos duplicados, lo que supone un porcentaje de 0.0% de los datos.
Hay 0 columnas con datos nulos, y son:
[]
y sin nulos hay 50 columnas y son:
['track_name', 'track_id', 'track_popularity', 'track_number', 'explicit', 'available_markets', 'artists_names', 'artists_ids', 'album_id', 'main_artist_id', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_sec', 'time_signature', 'album_name', 'album_release_date', 'total_tracks', 'type', 'image_url', 'album_popularity', 'album_label', 'followers', 'genres', 'artist_name', 'artist_popularity', 'lowest position', 'mean_position', 'position_std', 'best_position', 'times_in_rating', 'born_or_founded_in', 'positions_and_years_data', 'dj_score', 'release_year', 'release_month', 'track_name_length', 'main_artist_name_length', 'album_name_length', 'available_markets_count', 'artists_count', 'cover_id'

,% nulos,% no_nulos,tipo_dato,num_valores_unicos
track_name,0.0%,100.0%,object,37892
track_id,0.0%,100.0%,object,57461
track_popularity,0.0%,100.0%,float64,91
track_number,0.0%,100.0%,float64,50
explicit,0.0%,100.0%,bool,2


Principales estadísticos de las columnas categóricas:


,count,unique,top,freq
track_name,57461,37892,Follow Me,39
track_id,57461,57461,6mIrY9axk9DkBCk4eHXL6c,1
available_markets,57461,1203,"['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT...",25331
artists_names,57461,15614,['ATB'],651
artists_ids,57461,15884,['7jZM5w05mGhw6wTB1okhD9'],651
album_id,57461,16163,1nmnTaK967MrFHr6Jqs8xL,50
main_artist_id,57461,354,7jZM5w05mGhw6wTB1okhD9,1095
album_name,57461,12614,Brotherhood,215
album_release_date,57461,3235,2014-01-01,316
type,57461,1,album,57461


Principales estadísticos de las columnas numéricas:


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
track_popularity,57461.0,NaN,NaN,NaN,15.242982,16.363286,0.0,1.0,10.0,25.0,95.0
track_number,57461.0,NaN,NaN,NaN,6.443588,6.681924,1.0,2.0,4.0,9.0,50.0
explicit,57461,2,False,55454,NaN,NaN,NaN,NaN,NaN,NaN,NaN
danceability,57461.0,NaN,NaN,NaN,0.63094,0.140978,0.0,0.547,0.642,0.734,0.989
energy,57461.0,NaN,NaN,NaN,0.78375,0.173366,0.00002,0.695,0.827,0.916,1.0
key,57461.0,NaN,NaN,NaN,5.537965,3.601882,0.0,2.0,6.0,9.0,11.0
loudness,57461.0,NaN,NaN,NaN,-6.996427,3.270797,-48.12,-8.57,-6.523,-4.785,1.585
mode,57461.0,NaN,NaN,NaN,0.503106,0.499995,0.0,0.0,1.0,1.0,1.0
speechiness,57461.0,NaN,NaN,NaN,0.077542,0.069601,0.0,0.0414,0.0541,0.0814,0.934
acousticness,57461.0,NaN,NaN,NaN,0.074228,0.167963,0.000001,0.00128,0.00853,0.0527,0.995


,% nulos,% no_nulos,tipo_dato,num_valores_unicos
track_name,0.0%,100.0%,object,37892
track_id,0.0%,100.0%,object,57461
track_popularity,0.0%,100.0%,float64,91
track_number,0.0%,100.0%,float64,50
explicit,0.0%,100.0%,bool,2
available_markets,0.0%,100.0%,object,1203
artists_names,0.0%,100.0%,object,15614
artists_ids,0.0%,100.0%,object,15884
album_id,0.0%,100.0%,object,16163
main_artist_id,0.0%,100.0%,object,354


In [47]:
# Viendo el mínimo y máximo de release_year. 1900 no es correcto. Cambiar por 1990.
print(df2['release_year'].min())
print(df2['release_year'].max())

1900
2022


In [57]:
unique_years = sorted(df2['release_year'].unique(), reverse=True)

In [ ]:
sorted(map(int, unique_years), reverse=True)

In [69]:
if not df2[df2['release_year'] == 1900].empty:
    df2.loc[df2['release_year'] == 1900, 'release_year'] = 1990


In [71]:
df2['release_year'].unique()

array([2020, 2019, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009,
       2008, 2022, 2021, 2018, 2007, 2005, 2004, 2003, 2006, 2000, 1999,
       1998, 2002, 1994, 1993, 1992, 1995, 2001, 1997, 1991, 1990, 1996,
       1959, 1989, 1987])

In [75]:
filtered_rows = df2[df2['release_year'] == 1959]

In [77]:
#Borrando las filas donde el release year es 1959, ya que el género es blues, no electrónica.
df2 = df2.drop(filtered_rows.index)

In [85]:
# index=False es útil si solo quieres los datos y no necesitas la columna de índice.
df2.to_csv("archivos/Electro_music_on_Spotify.csv", index=False)